# Cyclic convolution

Given prime $q$, and a power of two $n = 2^k$, define the polynomial ring:

$$
R \leftarrow \mathbb{Z}_q[x] / \langle x^n - 1 \rangle
$$

A polynomial $\mathbf{a}$ in this polynomial ring can be represented using a set of $n$ coefficients:

$$
\mathbf{a} = \sum_{i=0}^{n-1}a_ix^i
$$

Where $n \mid q-1$, there exists an n-th primitive root $\omega_n$ in the multiplicative group $\mathbb{Z}_q^\ast$ (how to find it?). Using $\omega_n$ we can define the forward number theoretic transformation of the polynomial $\mathbf{a}$ from the time domain to the frequency domain:

$$
\hat{\mathbf{a}} \leftarrow \mathop{\text{NTT}}(\mathbf{a})
$$

where for $0 \leq j < n$:

$$
\hat{a_j} = \sum_{i=0}^{n-1}a_i \omega_n^{ij}
$$

Note that coefficient addition and multiplication are defined within the integer ring $\mathbb{Z}_q$.

The inverse transformation:

$$
a_j \leftarrow n^{-1}\sum_{i=0}^{n-1}\hat{a}_i\omega_n^{-ij}
$$

The frequency domain representation of a polynomial is useful because it makes polynomial multiplication in this quotient ring more efficient:

$$
\hat{\mathbf{c}} = \langle \hat{\mathbf{a}}, \hat{\mathbf{b}} \rangle
$$

Where the inner product is defined as coefficient-wise multiplication within $\mathbb{Z}_q$

In [1]:
from sympy.ntheory import isprime
from sympy.ntheory.residue_ntheory import primitive_root

def generate_params(nbits: int, qbits: int):
    """Generate n, q such that n is a power of 2, q is a prime, and n divides
    q - 1
    """
    n = 2 ** nbits
    q = (2 ** qbits // n + 1) * n + 1
    i = 0
    while not isprime(q):
        q += n
    return n, q

def nth_primitive_root(q: int, n: int):
    """Return the n-th primitive root modulus q"""
    return pow(primitive_root(q), (q-1) // n, q)

n, q = generate_params(8, 20)
assert isprime(q) and ((q - 1) % n == 0)
w = nth_primitive_root(q, n)

In [2]:
# Coefficient at lower index correspond to indeterminant of higher power
import random
from sympy import Poly, GF, symbols

a_coeffs = [random.randint(0, q-1) for _ in range(n)]
a_poly = Poly(a_coeffs, symbols("x"), domain=GF(q))

b_coeffs = [random.randint(0, q-1) for _ in range(n)]
b_poly = Poly(b_coeffs, symbols("x"), domain=GF(q))

modulus_coeffs = [0 for _ in range(n)]
modulus_coeffs[0] = 1
modulus_coeffs[-1] = -1
modulus_poly = Poly(modulus_coeffs, symbols("x"), domain=GF(q))

In [3]:
def ntt(n: int, time_coeffs: list[int], omega: int) -> list[int]:
    """Transform from time-domain coefficients import frequency-domain 
    coefficients
    """
    freq_coeffs = [0 for _ in range(n)]

    for j in range(n):
        sum_ = 0
        for i in range(n):
            sum_ += time_coeffs[n - i - 1] * pow(omega, i * j, q)
        freq_coeffs[n - j -1] = sum_ % q

    return freq_coeffs

def ntt_inv(n: int, freq_coeffs: list[int], omega: int) -> list[int]:
    """Transform from frequency-domain coefficients into time-domain 
    coefficients
    """
    time_coeffs = [0 for _ in range(n)]

    for j in range(n):
        sum_ = 0
        for i in range(n):
            sum_ += freq_coeffs[n - i - 1] * pow(omega, -i * j, q)
        time_coeffs[n - j - 1] = (sum_ * pow(n, -1, q)) % q

    return time_coeffs

assert ntt_inv(n, ntt(n, a_coeffs, w), w) == a_coeffs

In [4]:
a_coeffs_freqs = ntt(n, a_coeffs, w)
b_coeffs_freqs = ntt(n, b_coeffs, w)

c_coeffs_freqs = [a * b for (a, b) in zip(a_coeffs_freqs, b_coeffs_freqs)]
c_coeffs = ntt_inv(n, c_coeffs_freqs, w)
c_poly = Poly(c_coeffs, symbols("x"), domain=GF(q))

# TODO: this doesn't quite work just yet
display(c_poly, (a_poly * b_poly) % modulus_poly)

Poly(108000*x**255 - 45366*x**254 + 522183*x**253 - 194758*x**252 - 49084*x**251 + 240748*x**250 + 208704*x**249 + 455123*x**248 - 256551*x**247 - 291531*x**246 + 457684*x**245 - 267445*x**244 + 164458*x**243 - 442714*x**242 - 159208*x**241 + 254747*x**240 - 198689*x**239 - 57575*x**238 - 130690*x**237 - 382489*x**236 - 420750*x**235 - 356278*x**234 - 13318*x**233 + 260171*x**232 + 111899*x**231 + 284513*x**230 - 510081*x**229 - 17924*x**228 - 228399*x**227 + 12662*x**226 - 263799*x**225 - 313227*x**224 + 352638*x**223 + 211782*x**222 - 350590*x**221 + 438810*x**220 - 223685*x**219 + 304552*x**218 + 59499*x**217 - 446590*x**216 + 91771*x**215 + 400371*x**214 + 376446*x**213 - 167431*x**212 - 186861*x**211 - 460238*x**210 - 99029*x**209 + 428069*x**208 - 186857*x**207 + 492097*x**206 - 359501*x**205 - 22434*x**204 + 348089*x**203 + 178939*x**202 + 502799*x**201 - 90782*x**200 + 403456*x**199 - 270266*x**198 + 263582*x**197 + 167521*x**196 - 95357*x**195 - 106168*x**194 - 88038*x**193 - 

Poly(-347117*x**254 - 34666*x**253 + 375526*x**252 - 236213*x**251 - 392507*x**250 - 443045*x**249 + 137498*x**248 + 60209*x**247 - 346541*x**246 + 261555*x**245 - 219769*x**244 + 217711*x**243 + 255394*x**242 - 458749*x**241 - 306776*x**240 + 355660*x**239 - 92958*x**238 + 67239*x**237 + 86948*x**236 + 313442*x**235 + 220751*x**234 - 208002*x**233 + 280756*x**232 - 309024*x**231 - 436446*x**230 + 350566*x**229 - 294867*x**228 - 233185*x**227 + 13567*x**226 - 19299*x**225 + 404812*x**224 + 523228*x**223 - 67097*x**222 + 489685*x**221 + 523558*x**220 - 318328*x**219 - 335871*x**218 - 479305*x**217 + 270226*x**216 - 390940*x**215 - 333076*x**214 - 110320*x**213 - 292890*x**212 - 291314*x**211 + 492122*x**210 + 489181*x**209 - 255737*x**208 - 344034*x**207 + 326640*x**206 - 282766*x**205 + 193528*x**204 + 421200*x**203 - 192910*x**202 - 234550*x**201 + 384423*x**200 + 115254*x**199 - 378252*x**198 - 392390*x**197 - 35162*x**196 + 236417*x**195 - 171278*x**194 - 236409*x**193 - 61827*x**19